In [5]:
# Importa as bibliotecas necessárias
import wget
import pandas as pd
import json
import datetime
from pandas.io.json import json_normalize

In [27]:
# Define os dados para download
url = 'http://plataforma.saude.gov.br/novocoronavirus/resources/scripts/'
arquivo = 'database.js'

In [ ]:
# Baixa o arquivo JS
wget.download(url + arquivo, arquivo)

In [28]:
# Converte o arquivo JavaScript para JSON
base_js = open(arquivo, "rt")
base_json = open("database.json", "wt")

for line in base_js:
	base_json.write(line.replace('var database=', ''))

base_js.close()
base_json.close()

In [34]:
# Exporta o dataframe
df = json.load(open('database.json'))
df = json_normalize(data=df['brazil'], record_path='values',meta=['date','time'])
df.uid[df.date == '18/02/2020']

145    23
146    31
147    33
148    35
149    41
150    42
151    43
152    53
Name: uid, dtype: object

In [46]:
# Insere o nome dos estados
indice = pd.read_csv('indice.csv')

# Transforma a ID em string
df.uid = df.uid.astype(str)
indice.uid = indice.uid.astype(str)

# Junta as UFs nos dados
df = pd.merge(df, indice, on='uid', how='left')

In [47]:
# Transforma números em inteiros
dados_numericos = ['suspects', 'refuses', 'confirmado', 'deads','cases']

for coluna in dados_numericos:
    df[coluna] = df[coluna].fillna(0)
    df[coluna] = df[coluna].astype(int)

In [48]:
# Exporta a base em CSV com a data da execução no nome do arquivo
agora = str(datetime.date.today()).replace(" ", "").replace(":", "-").replace(".", "-")
dados = 'dados/coronabr_' + agora + '.csv'
df.to_csv(dados, index = False)